In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from tqdm import tqdm
import pandas as pd
import numpy as np
from pathlib import Path

In [3]:
from exact_sync.v1.api.annotations_api import AnnotationsApi
from exact_sync.v1.api.images_api import ImagesApi
from exact_sync.v1.api.image_sets_api import ImageSetsApi
from exact_sync.v1.api.annotation_types_api import AnnotationTypesApi
from exact_sync.v1.api.products_api import ProductsApi
from exact_sync.v1.api.teams_api import TeamsApi

from exact_sync.v1.models import ImageSet, Team, Product, AnnotationType, Image, Annotation, AnnotationMediaFile
from exact_sync.v1.rest import ApiException
from exact_sync.v1.configuration import Configuration
from exact_sync.v1.api_client import ApiClient

In [4]:
configuration = Configuration()
configuration.username = 'marzahl'
configuration.password = 'imagetagger'
configuration.host =  "https://exact.cs.fau.de/"

image_set_ids = [222]

In [5]:
client = ApiClient(configuration)

image_sets_api = ImageSetsApi(client)
annotations_api = AnnotationsApi(client)
annotation_types_api = AnnotationTypesApi(client)
images_api = ImagesApi(client)
product_api = ProductsApi(client)

In [8]:
screened = [
    'BAL_Cat_Turnbull blue_1.svs',
    'BAL_Cat_Turnbull blue_2.svs',
    'BAL_Cat_Turnbull blue_6.svs',
    'BAL_Cat_Turnbull blue_10.svs',
    'BAL_Cat_Turnbull blue_12.svs',
    'BAL_Cat_Turnbull blue_13.svs',
    'BAL_Cat_Turnbull blue_14.svs',
    
    'BAL_Cat_Turnbull blue_1.tiff',
    'BAL_Cat_Turnbull blue_2.tiff',
    'BAL_Cat_Turnbull blue_6.tiff',
    'BAL_Cat_Turnbull blue_10.tiff',
    'BAL_Cat_Turnbull blue_12.tiff',
    'BAL_Cat_Turnbull blue_13.tiff',
    'BAL_Cat_Turnbull blue_14.tiff',
]

In [7]:
images = {}
annotation_types = {}

for image_set in image_set_ids:
    for image in images_api.list_images(pagination=False, image_set=image_set).results:
        
        if image.name not in screened:
            continue
        
        images[image.id] = image
    for product in product_api.list_products(imagesets=image_set).results:
        for anno_type in annotation_types_api.list_annotation_types(product=product.id).results:
            annotation_types[anno_type.id] = anno_type

In [8]:
rows = []
for image_id, image in tqdm(images.items()):

    for annotation_type_id, annotation_type in annotation_types.items():

        for anno in annotations_api.list_annotations(annotation_type=annotation_type_id, image=image_id, pagination=False, fields="id,unique_identifier,annotation_type,vector,user,deleted,last_editor").results:
            
            species = "Cat"
            data_set_name = "SDATA"
            version = "Inference"
            
            # anno_id, image_id, image_name, anno_type_name, vector, user_id, user_name
            image_type = "Prussian" if "erliner" in image.name else "Turnbull"
            if anno.vector is not None and anno.user in [1, 11, 12]:
                rows.append([anno.id, image_id, image.image_set, species, image.name, image_type, int(annotation_type.name), anno.vector, anno.unique_identifier, anno.user, anno.deleted, anno.last_editor, data_set_name, version])

    df = pd.DataFrame(rows, columns=['id', 'image_id', 'image_set', 'species' ,'image_name', 'image_type', 'grade', 'vector', 'unique_identifier', 'user_id', 'deleted', 'last_editor', 'data_set_name', 'version'])
    df.to_pickle("EIPH_Cat_SDATA_Inference_Annotations.pkl")

100%|████████████████████████████████████████████████████████████████████████████████████| 7/7 [11:17<00:00, 96.83s/it]


In [11]:
rows = []
for image_set in [249]:
    for image in tqdm(images_api.list_images(pagination=False, image_set=image_set).results):
        
        if image.name not in screened:
            continue

        for annotation_type_id, annotation_type in annotation_types.items():

            for anno in annotations_api.list_annotations(annotation_type=annotation_type_id, 
                                                         image=image.id, pagination=False, deleted=False,
                                                         fields="id,unique_identifier,annotation_type,vector,user,deleted,last_editor").results:

                species = "Cat"
                data_set_name = "SDATA"
                version = "Cluster"

                # anno_id, image_id, image_name, anno_type_name, vector, user_id, user_name
                image_type = "Prussian" if "erliner" in image.name else "Turnbull"
                if anno.vector is not None and anno.user in [1, 11, 12]:
                    rows.append([anno.id, image.id, image.image_set, species, image.name, image_type, int(annotation_type.name), anno.vector, anno.unique_identifier, anno.user, anno.deleted, anno.last_editor, data_set_name, version])

        df = pd.DataFrame(rows, columns=['id', 'image_id', 'image_set', 'species' ,'image_name', 'image_type', 'grade', 'vector', 'unique_identifier', 'user_id', 'deleted', 'last_editor', 'data_set_name', 'version'])
        df.to_pickle("EIPH_Cat_SDATA_Cluster_Annotations.pkl")

100%|██████████████████████████████████████████████████████████████████████████████████| 14/14 [10:54<00:00, 46.75s/it]


In [10]:
rows = []
for image_id, image in tqdm(images.items()):

    for annotation_type_id, annotation_type in annotation_types.items():

        for anno in annotations_api.list_annotations(annotation_type=annotation_type_id, 
                                                     image=image_id, pagination=False, deleted=False,
                                                     fields="id,unique_identifier,annotation_type,vector,user,deleted,last_editor").results:
            
            species = "Cat"
            data_set_name = "SDATA"
            version = "Screened"
            
            # anno_id, image_id, image_name, anno_type_name, vector, user_id, user_name
            image_type = "Prussian" if "erliner" in image.name else "Turnbull"
            if anno.vector is not None and anno.user in [1, 11, 12]:
                rows.append([anno.id, image_id, image.image_set, species, image.name, image_type, int(annotation_type.name), anno.vector, anno.unique_identifier, anno.user, anno.deleted, anno.last_editor, data_set_name, version])

    df = pd.DataFrame(rows, columns=['id', 'image_id', 'image_set', 'species' ,'image_name', 'image_type', 'grade', 'vector', 'unique_identifier', 'user_id', 'deleted', 'last_editor', 'data_set_name', 'version'])
    df.to_pickle("EIPH_Cat_SDATA_Screening_Annotations.pkl")

100%|████████████████████████████████████████████████████████████████████████████████████| 7/7 [02:29<00:00, 21.43s/it]


In [11]:
rows = []
for image_set in [284]:
    for image in tqdm(images_api.list_images(pagination=False, image_set=image_set).results):

        for annotation_type_id, annotation_type in annotation_types.items():

            for anno in annotations_api.list_annotations(annotation_type=annotation_type_id, 
                                                         image=image.id, pagination=False, deleted=False,
                                                         fields="id,unique_identifier,annotation_type,vector,user,deleted,last_editor").results:

                species = "Cat"
                data_set_name = "SDATA"
                version = "Density"

                # anno_id, image_id, image_name, anno_type_name, vector, user_id, user_name
                image_type = "Prussian" if "erliner" in image.name else "Turnbull"
                if anno.vector is not None and anno.user in [1, 11, 12]:
                    rows.append([anno.id, image.id, image.image_set, species, image.name, image_type, int(annotation_type.name), anno.vector, anno.unique_identifier, anno.user, anno.deleted, anno.last_editor, data_set_name, version])

        df = pd.DataFrame(rows, columns=['id', 'image_id', 'image_set', 'species' ,'image_name', 'image_type', 'grade', 'vector', 'unique_identifier', 'user_id', 'deleted', 'last_editor', 'data_set_name', 'version'])
        df.to_pickle("EIPH_Cat_SDATA_Density_Annotations.pkl")

100%|████████████████████████████████████████████████████████████████████████████████████| 7/7 [01:36<00:00, 13.75s/it]
